
---

# **Chapter 18: GoRouter & Advanced Navigation**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Set up and configure GoRouter for declarative navigation
- Implement ShellRoute for nested navigation (bottom navigation, tabs, drawers)
- Create redirects and guards for authentication and authorization
- Handle query parameters and path parameters with GoRouter
- Implement error handling and custom 404 pages
- Use GoRouter's advanced features (logging, observers, custom transitions)
- Integrate GoRouter with state management solutions

---

## **Prerequisites**

- Completed Chapter 16: Navigation Fundamentals
- Completed Chapter 17: Deep Linking & Navigation 2.0
- Understanding of Navigator 2.0 concepts
- Knowledge of BuildContext and widget lifecycle
- Basic familiarity with state management patterns

---

## **18.1 Introduction to GoRouter**

GoRouter is a declarative routing package for Flutter that simplifies Navigator 2.0 with a cleaner API and built-in support for common navigation patterns.

### **Why GoRouter?**

```dart
import 'package:flutter/material.dart';
import 'package:go_router/go_router.dart';

// ============================================
// NAVIGATOR 2.0 VS GOROUTER
// ============================================
// Navigator 2.0 requires implementing:
// - RouterDelegate
// - RouteInformationParser
// - Navigation state management
// - Manual URL parsing and updating
// This results in significant boilerplate code

// GoRouter simplifies this with:
// - Declarative route definitions
// - Automatic URL parsing and updating
// - Built-in deep linking support
// - Less boilerplate code
// - Easier to maintain and understand

// ============================================
// BASIC GOROUTER SETUP
// ============================================

void main() {
  runApp(const GoRouterApp());
}

class GoRouterApp extends StatelessWidget {
  // Define the GoRouter configuration
  // This router will be used throughout the app
  
  final GoRouter _router = GoRouter(
    // Initial location: the route the app starts on
    // This is the initial URL/route when the app launches
    initialLocation: '/',
    
    // Debug logging: prints navigation events to the console
    // Useful for debugging navigation flow
    debugLogDiagnostics: true,
    
    // Error builder: shown when navigation fails (e.g., 404)
    // This is your custom error page
    errorBuilder: (context, state) => ErrorScreen(error: state.error),
    
    // Routes: define all the routes in your app
    // GoRouter uses a tree structure for nested routes
    routes: [
      // ============================================
      // TOP-LEVEL ROUTE: HOME
      // ============================================
      GoRoute(
        // The path for this route
        // This is the URL path that maps to this route
        path: '/',
        
        // The name for this route (used for programmatic navigation)
        // Instead of using the path, you can use this name to navigate
        name: 'home',
        
        // The widget to display for this route
        // This is the screen that appears when this route is active
        builder: (context, state) => const HomeScreen(),
      ),
      
      // ============================================
      // TOP-LEVEL ROUTE: ABOUT
      // ============================================
      GoRoute(
        path: '/about',
        name: 'about',
        builder: (context, state) => const AboutScreen(),
      ),
      
      // ============================================
      // TOP-LEVEL ROUTE: PROFILE
      // ============================================
      GoRoute(
        path: '/profile',
        name: 'profile',
        builder: (context, state) => const ProfileScreen(),
      ),
      
      // ============================================
      // ROUTE WITH PATH PARAMETER: PRODUCT
      // ============================================
      // Path parameters are defined with a leading colon (:)
      // Example URL: /product/123 matches this route
      // The parameter value is accessible via state.params
      GoRoute(
        path: '/product/:id',
        name: 'product',
        builder: (context, state) {
          // Extract the path parameter from state.pathParameters
          // state.pathParameters is a Map<String, String> containing
          // all the path parameters defined in the route path
          
          final productId = state.pathParameters['id'] ?? 'unknown';
          
          return ProductScreen(productId: productId);
        },
      ),
      
      // ============================================
      // ROUTE WITH MULTIPLE PATH PARAMETERS
      // ============================================
      // You can have multiple path parameters in a single route
      // Example URL: /category/electronics/product/123
      GoRoute(
        path: '/category/:categoryId/product/:productId',
        name: 'category-product',
        builder: (context, state) {
          final categoryId = state.pathParameters['categoryId'] ?? 'unknown';
          final productId = state.pathParameters['productId'] ?? 'unknown';
          
          return CategoryProductScreen(
            categoryId: categoryId,
            productId: productId,
          );
        },
      ),
      
      // ============================================
      // ROUTE WITH QUERY PARAMETERS: SEARCH
      // ============================================
      // Query parameters are defined in the URL after a ?
      // Example URL: /search?q=flutter&sort=recent
      // Query parameters are accessible via state.uri.queryParameters
      GoRoute(
        path: '/search',
        name: 'search',
        builder: (context, state) {
          // Extract query parameters from state.uri.queryParameters
          // state.uri is the full URI for the current route
          // queryParameters is a Map<String, String> containing
          // all the query parameters from the URL
          
          final query = state.uri.queryParameters['q'] ?? '';
          final sort = state.uri.queryParameters['sort'] ?? 'relevance';
          
          return SearchScreen(
            query: query,
            sort: sort,
          );
        },
      ),
      
      // ============================================
      // ROUTE WITH BOTH PATH AND QUERY PARAMETERS
      // ============================================
      // You can combine path and query parameters
      // Example URL: /product/123?ref=homepage&source=banner
      GoRoute(
        path: '/product-details/:id',
        name: 'product-details',
        builder: (context, state) {
          final productId = state.pathParameters['id'] ?? 'unknown';
          final referrer = state.uri.queryParameters['ref'];
          final source = state.uri.queryParameters['source'];
          
          return ProductDetailsScreen(
            productId: productId,
            referrer: referrer,
            source: source,
          );
        },
      ),
    ],
  );

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'GoRouter Demo',
      debugShowCheckedModeBanner: false,
      
      // Theme configuration
      theme: ThemeData(
        useMaterial3: true,
        colorSchemeSeed: Colors.blue,
      ),
      
      // Pass the GoRouter to MaterialApp.router
      // This is the key integration point between GoRouter and MaterialApp
      routerConfig: _router,
      // routerConfig is a shortcut that sets:
      // - routerDelegate
      // - routeInformationParser
      // - routeInformationProvider
      // - backButtonDispatcher
    );
  }
}

// ============================================
// SCREEN WIDGETS
// ============================================

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Home')),
      body: ListView(
        children: [
          ListTile(
            title: const Text('About'),
            onTap: () {
              // Navigate to the about route using the path
              // context.go() navigates to the specified route
              // and replaces the current route in the stack
              context.go('/about');
            },
          ),
          ListTile(
            title: const Text('Profile'),
            onTap: () {
              // Navigate using the route name
              // GoRouter names are defined in the GoRoute configuration
              context.goNamed('profile');
            },
          ),
          ListTile(
            title: const Text('Product 123'),
            subtitle: const Text('Path parameter'),
            onTap: () {
              // Navigate with a path parameter
              // Use context.go() and build the URL with the parameter
              context.go('/product/123');
            },
          ),
          ListTile(
            title: const Text('Category Product'),
            subtitle: const Text('Multiple path parameters'),
            onTap: () {
              // Navigate with multiple path parameters
              context.go('/category/electronics/product/456');
            },
          ),
          ListTile(
            title: const Text('Search Results'),
            subtitle: const Text('Query parameters'),
            onTap: () {
              // Navigate with query parameters
              // Query parameters are added to the URL after ?
              context.go('/search?q=flutter&sort=recent');
            },
          ),
          ListTile(
            title: const Text('Product Details'),
            subtitle: const Text('Path and query parameters'),
            onTap: () {
              // Navigate with both path and query parameters
              context.go('/product-details/789?ref=homepage&source=banner');
            },
          ),
        ],
      ),
    );
  }
}

class AboutScreen extends StatelessWidget {
  const AboutScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('About')),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.go('/'),
          child: const Text('Go Home'),
        ),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Profile')),
      body: const Center(
        child: Text('Profile Screen'),
      ),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String productId;
  
  const ProductScreen({super.key, required this.productId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Product: $productId')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Product ID: $productId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}

class CategoryProductScreen extends StatelessWidget {
  final String categoryId;
  final String productId;
  
  const CategoryProductScreen({
    super.key,
    required this.categoryId,
    required this.productId,
  });

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Category: $categoryId, Product: $productId'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.go('/'),
          child: const Text('Go Home'),
        ),
      ),
    );
  }
}

class SearchScreen extends StatelessWidget {
  final String query;
  final String sort;
  
  const SearchScreen({
    super.key,
    required this.query,
    required this.sort,
  });

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Search: $query'),
        actions: [
          // Display the sort parameter
          Center(
            child: Padding(
              padding: const EdgeInsets.symmetric(horizontal: 16),
              child: Text('Sort: $sort'),
            ),
          ),
        ],
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Query: $query'),
            Text('Sort: $sort'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}

class ProductDetailsScreen extends StatelessWidget {
  final String productId;
  final String? referrer;
  final String? source;
  
  const ProductDetailsScreen({
    super.key,
    required this.productId,
    this.referrer,
    this.source,
  });

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Product Details: $productId')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Product ID: $productId'),
            if (referrer != null) Text('Referrer: $referrer'),
            if (source != null) Text('Source: $source'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}

class ErrorScreen extends StatelessWidget {
  final Exception? error;
  
  const ErrorScreen({super.key, this.error});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Error')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(Icons.error, size: 64, color: Colors.red),
            const SizedBox(height: 16),
            Text(error?.toString() ?? 'An error occurred'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **GoRouter simplification**: GoRouter eliminates the need to implement `RouterDelegate`, `RouteInformationParser`, and manual navigation state management. Instead, you define routes declaratively using `GoRoute` objects.

- **`GoRouter` configuration**: The `GoRouter` constructor takes several key parameters:
  - `initialLocation`: The route the app starts on (e.g., `'/'`). This determines the initial URL when the app launches.
  - `debugLogDiagnostics`: When `true`, prints navigation events to the console for debugging. This includes route changes, parameter extraction, and error messages.
  - `errorBuilder`: A function that builds an error screen when navigation fails (e.g., 404 for unknown routes). The function receives the `BuildContext` and the `GoRouterState` containing the error.
  - `routes`: A list of `GoRoute` objects defining all routes in the app.

- **`GoRoute` properties**: Each route has:
  - `path`: The URL path that maps to this route (e.g., `'/product/:id'`). This is what appears in the browser URL (on web) or deep link.
  - `name`: An optional name for the route, used for programmatic navigation (e.g., `'product'`). Using names instead of paths makes navigation more maintainable.
  - `builder`: A function that builds the widget for this route. It receives the `BuildContext` and `GoRouterState`, which contains navigation information like path parameters and query parameters.

- **Path parameters**: Defined in the route path with a leading colon (`:`). For example, `'/product/:id'` matches URLs like `/product/123`, where `123` is the value of the `id` parameter. Path parameters are accessed via `state.pathParameters['paramName']`.

- **Query parameters**: Defined in the URL after a `?`. For example, `/search?q=flutter&sort=recent` has two query parameters: `q=flutter` and `sort=recent`. Query parameters are accessed via `state.uri.queryParameters['paramName']`.

- **`GoRouterState`**: Provided to the `builder` function, it contains:
  - `pathParameters`: Map of path parameters (e.g., `{'id': '123'}` for `/product/123`)
  - `uri`: The full URI for the current route, including query parameters
  - `path`: The path without query parameters
  - `name`: The name of the route (if defined)

- **Navigation methods**: GoRouter provides extension methods on `BuildContext`:
  - `context.go(path)`: Navigates to the specified path, replacing the current route in the stack. This is the default navigation method.
  - `context.goNamed(name, params)`: Navigates to the route with the specified name, optionally with path parameters.
  - `context.push(path)`: Pushes a new route onto the stack (like Navigator 1.0's `push`).
  - `context.pop()`: Pops the current route (like Navigator 1.0's `pop`).

- **`MaterialApp.router` with GoRouter**: Integration is simple - pass the `GoRouter` instance to the `routerConfig` parameter. This automatically sets up all the necessary Navigator 2.0 components.

- **`context.go()` vs. `context.push()`**:
  - `context.go()`: Replaces the current route in the stack. This is the default and most common navigation method in GoRouter.
  - `context.push()`: Pushes a new route onto the stack, allowing the user to go back. This is useful for modal dialogs or temporary screens.

- **Named navigation**: Using `context.goNamed('routeName')` is more maintainable than using paths, as you can change the path without updating all navigation calls throughout your app.

---

## **18.2 Navigation Methods in GoRouter**

GoRouter provides several navigation methods with different behaviors for managing the navigation stack.

### **Complete Navigation Methods Overview**

```dart
import 'package:flutter/material.dart';
import 'package:go_router/go_router.dart';

void main() {
  runApp(const NavigationMethodsApp());
}

class NavigationMethodsApp extends StatelessWidget {
  final GoRouter _router = GoRouter(
    initialLocation: '/',
    debugLogDiagnostics: true,
    routes: [
      GoRoute(
        path: '/',
        name: 'home',
        builder: (context, state) => const HomeScreen(),
      ),
      GoRoute(
        path: '/page1',
        name: 'page1',
        builder: (context, state) => const Page1Screen(),
      ),
      GoRoute(
        path: '/page2',
        name: 'page2',
        builder: (context, state) => const Page2Screen(),
      ),
      GoRoute(
        path: '/page3',
        name: 'page3',
        builder: (context, state) => const Page3Screen(),
      ),
      GoRoute(
        path: '/page4',
        name: 'page4',
        builder: (context, state) => const Page4Screen(),
      ),
      GoRoute(
        path: '/result',
        name: 'result',
        builder: (context, state) {
          final result = state.uri.queryParameters['result'] ?? 'No result';
          return ResultScreen(result: result);
        },
      ),
    ],
  );

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Navigation Methods',
      routerConfig: _router,
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Navigation Methods'),
        // Display the current location (URL) in the AppBar
        actions: [
          // Get the current location from GoRouter
          Builder(
            builder: (context) {
              final location = GoRouterState.of(context).uri.toString();
              return Padding(
                padding: const EdgeInsets.symmetric(horizontal: 16),
                child: Center(
                  child: Text(
                    location,
                    style: const TextStyle(fontSize: 12),
                  ),
                ),
              );
            },
          ),
        ],
      ),
      body: ListView(
        children: [
          // ============================================
          // CONTEXT.GO()
          // ============================================
          ListTile(
            title: const Text('context.go()'),
            subtitle: const Text('Replace current route'),
            onTap: () {
              // context.go() navigates to the specified route
              // It REPLACES the current route in the stack
              // This is the default navigation method in GoRouter
              // The navigation stack after this: [Page1]
              // The '/' route is removed from the stack
              
              context.go('/page1');
              
              print('After go(): Stack = [Page1]');
              print('Previous route (/) was replaced');
            },
          ),
          ListTile(
            title: const Text('context.goNamed()'),
            subtitle: const Text('Replace using route name'),
            onTap: () {
              // context.goNamed() navigates using the route name
              // It REPLACES the current route in the stack
              // Using names is more maintainable than using paths
              // The navigation stack after this: [Page2]
              
              context.goNamed('page2');
              
              print('After goNamed(): Stack = [Page2]');
              print('Previous route (/) was replaced');
            },
          ),
          
          const Divider(),
          
          // ============================================
          // CONTEXT.PUSH()
          // ============================================
          ListTile(
            title: const Text('context.push()'),
            subtitle: const Text('Push new route onto stack'),
            onTap: () {
              // context.push() pushes a new route onto the stack
              // It ADDS to the stack, allowing the user to go back
              // This is similar to Navigator.push() in Navigator 1.0
              // The navigation stack after this: [/, Page1]
              // The '/' route remains in the stack below Page1
              
              context.push('/page1');
              
              print('After push(): Stack = [/, Page1]');
              print('Can go back to (/)');
            },
          ),
          ListTile(
            title: const Text('context.pushNamed()'),
            subtitle: const Text('Push using route name'),
            onTap: () {
              // context.pushNamed() pushes using the route name
              // It ADDS to the stack
              // The navigation stack after this: [/, Page2]
              
              context.pushNamed('page2');
              
              print('After pushNamed(): Stack = [/, Page2]');
              print('Can go back to (/)');
            },
          ),
          
          const Divider(),
          
          // ============================================
          // CONTEXT.POP()
          // ============================================
          ListTile(
            title: const Text('context.pop()'),
            subtitle: const Text('Pop current route'),
            onTap: () {
              // context.pop() removes the current route from the stack
              // It returns to the previous route
              // This is similar to Navigator.pop() in Navigator 1.0
              // The navigation stack after this: [/]
              // The Page1 route is removed from the stack
              
              context.pop();
              
              print('After pop(): Stack = [/]');
              print('Returned to (/)');
            },
          ),
          ListTile(
            title: const Text('context.pop(result)'),
            subtitle: const Text('Pop with result'),
            onTap: () async {
              // Navigate to Page3 and wait for a result
              final result = await context.push<String>('/page3');
              
              print('Received result: $result');
              
              // Show the result in a snackbar
              if (context.mounted) {
                ScaffoldMessenger.of(context).showSnackBar(
                  SnackBar(content: Text('Result: $result')),
                );
              }
            },
          ),
          
          const Divider(),
          
          // ============================================
          // ADVANCED METHODS
          // ============================================
          ListTile(
            title: const Text('context.goNamed() with parameters'),
            subtitle: const Text('Named navigation with path parameters'),
            onTap: () {
              // Navigate with path parameters using goNamed
              // Parameters are passed as a map to the pathParameters argument
              // The route must define the path parameter in its path (e.g., :id)
              // Example route: /product/:id
              
              // context.goNamed('product', pathParameters: {'id': '123'});
              // This would navigate to /product/123
              
              // For this example, we'll just use goNamed without parameters
              context.goNamed('page3');
              
              print('After goNamed with params: Stack = [Page3]');
            },
          ),
          ListTile(
            title: const Text('context.pushNamed() with parameters'),
            subtitle: const Text('Push with path parameters'),
            onTap: () {
              // Navigate with path parameters using pushNamed
              // Parameters are passed as a map to the pathParameters argument
              
              context.pushNamed('page4');
              
              print('After pushNamed with params: Stack = [/, Page4]');
            },
          ),
          ListTile(
            title: const Text('context.pushNamed() with query parameters'),
            subtitle: const Text('Push with query parameters'),
            onTap: () {
              // Navigate with query parameters using pushNamed
              // Query parameters are passed as a map to the queryParameters argument
              // Query parameters appear in the URL after ?
              // Example: /result?result=success
              
              context.pushNamed('result', queryParameters: {'result': 'success'});
              
              print('After pushNamed with query params: Stack = [/, Result]');
              print('URL: /result?result=success');
            },
          ),
        ],
      ),
    );
  }
}

class Page1Screen extends StatelessWidget {
  const Page1Screen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Page 1')),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.pop(),
          child: const Text('Go Back (pop)'),
        ),
      ),
    );
  }
}

class Page2Screen extends StatelessWidget {
  const Page2Screen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Page 2')),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.pop(),
          child: const Text('Go Back (pop)'),
        ),
      ),
    );
  }
}

class Page3Screen extends StatelessWidget {
  const Page3Screen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Page 3')),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            // Pop with a result
            // The result is returned to the previous screen
            // The previous screen awaits the result from its push() call
            
            context.pop('Result from Page3');
          },
          child: const Text('Go Back with Result'),
        ),
      ),
    );
  }
}

class Page4Screen extends StatelessWidget {
  const Page4Screen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      AppBar(title: const Text('Page 4')),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.pop(),
          child: const Text('Go Back (pop)'),
        ),
      ),
    );
  }
}

class ResultScreen extends StatelessWidget {
  final String result;
  
  const ResultScreen({super.key, required this.result});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Result')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Result: $result'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.pop(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`context.go()`**: The primary navigation method in GoRouter. It navigates to the specified route and **replaces** the current route in the stack. This is different from Navigator 1.0's `push()`, which adds to the stack. Use `go()` when you want to replace the current screen (e.g., after login, after form submission, for tab navigation).

- **`context.goNamed()`**: Similar to `go()`, but uses the route name instead of the path. This is more maintainable because you can change the path without updating all navigation calls throughout your app. The name must be defined in the `GoRoute` configuration.

- **`context.push()`**: Pushes a new route onto the stack, adding to it (like Navigator 1.0's `push()`). The user can go back to the previous screen using the back button. Use `push()` when you want to show a temporary screen (e.g., a details page, a modal dialog, a secondary flow).

- **`context.pushNamed()`**: Similar to `push()`, but uses the route name. Provides the same maintainability benefits as `goNamed()`.

- **`context.pop()`**: Removes the current route from the stack and returns to the previous route. This is similar to Navigator 1.0's `pop()`. You can optionally pass a result back to the previous screen using `context.pop(result)`.

- **Receiving results**: When you use `context.push<String>()` with a type parameter, it returns a `Future` that completes when the route is popped. The result is the value passed to `context.pop(result)`. This allows you to pass data back to the previous screen.

- **Path parameters with named navigation**: Use `context.goNamed('routeName', pathParameters: {'paramName': 'value'})` to navigate with path parameters using the route name. This is type-safe and maintainable.

- **Query parameters with named navigation**: Use `context.pushNamed('routeName', queryParameters: {'paramName': 'value'})` to navigate with query parameters. Query parameters appear in the URL after `?` and are accessible via `state.uri.queryParameters` in the destination screen.

- **Navigation stack visualization**:
  - `context.go('/page1')`: Stack goes from `[/]` to `[Page1]` (replacement)
  - `context.push('/page1')`: Stack goes from `[/]` to `[/, Page1]` (addition)
  - `context.pop()`: Stack goes from `[/, Page1]` to `[/]` (removal)

- **Choosing between `go()` and `push()`**:
  - Use `go()` when: You want to replace the current screen (e.g., after login, tab navigation, form submission)
  - Use `push()` when: You want to show a temporary screen (e.g., details page, modal dialog, secondary flow)

- **`GoRouterState.of(context)`**: Accesses the current router state, which contains information like the current URI, path parameters, and query parameters. This is useful for displaying the current location in the UI.

---

## **18.3 ShellRoute for Nested Navigation**

ShellRoute is a powerful GoRouter feature for implementing nested navigation, such as bottom navigation bars, tabs, and drawers.

### **Bottom Navigation with ShellRoute**

```dart
import 'package:flutter/material.dart';
import 'package:go_router/go_router.dart';

void main() {
  runApp(const ShellRouteApp());
}

class ShellRouteApp extends StatelessWidget {
  // ============================================
  // GOROUTER WITH SHELLROUTE
  // ============================================
  
  final GoRouter _router = GoRouter(
    initialLocation: '/home',
    debugLogDiagnostics: true,
    
    routes: [
      // ============================================
      // SHELLROUTE FOR BOTTOM NAVIGATION
      // ============================================
      // ShellRoute wraps child routes with a common layout
      // This is perfect for bottom navigation bars, tabs, or drawers
      // The shell remains visible while the inner routes change
      
      ShellRoute(
        // The path for this shell route
        // This is the base path for all child routes
        // Child routes are relative to this path
        path: '/',
        
        // The builder creates the shell widget (bottom navigation bar)
        // It receives a child widget that represents the current inner route
        // The shell can wrap this child with additional UI (e.g., bottom nav bar)
        
        builder: (context, state, child) {
          // child is the widget for the current inner route
          // This is what changes as the user navigates within the shell
          
          return MainScaffold(child: child);
        },
        
        // The child routes that are displayed within the shell
        // These routes can be navigated to, and the shell remains visible
        
        routes: [
          // ============================================
          // HOME TAB
          // ============================================
          GoRoute(
            // This route is displayed within the shell
            // The full path is /home (shell path / + this route path)
            path: 'home',
            
            // The name for programmatic navigation
            name: 'home',
            
            // PageBuilder allows custom page configuration
            // This is useful for setting page transitions or titles
            pageBuilder: (context, state) {
              return MaterialPage(
                key: state.pageKey,
                // state.pageKey is a unique key for this page
                // It helps Flutter identify and reuse pages
                
                child: const HomeScreen(),
                // This is the widget displayed within the shell
                // The shell (MainScaffold) wraps this with the bottom nav bar
              );
            },
          ),
          
          // ============================================
          // SEARCH TAB
          // ============================================
          GoRoute(
            path: 'search',
            name: 'search',
            pageBuilder: (context, state) {
              return MaterialPage(
                key: state.pageKey,
                child: const SearchScreen(),
              );
            },
          ),
          
          // ============================================
          // PROFILE TAB
          // ============================================
          GoRoute(
            path: 'profile',
            name: 'profile',
            pageBuilder: (context, state) {
              return MaterialPage(
                key: state.pageKey,
                child: const ProfileScreen(),
              );
            },
          ),
          
          // ============================================
          // NESTED ROUTE: PRODUCT DETAILS
          // ============================================
          // This is a child route of the shell
          // It's displayed within the shell (bottom nav bar remains visible)
          // However, it's a deeper route than the top-level tabs
          
          GoRoute(
            path: 'product/:id',
            name: 'product',
            pageBuilder: (context, state) {
              final productId = state.pathParameters['id'] ?? 'unknown';
              
              return MaterialPage(
                key: state.pageKey,
                child: ProductScreen(productId: productId),
              );
            },
            
            // Additional child routes can be nested here
            // For example, product reviews, related products, etc.
          ),
          
          // ============================================
          // NESTED ROUTE: SETTINGS
          // ============================================
          // This is shown as a full-screen page, hiding the bottom nav bar
          // We can achieve this by using a parent route without the shell
          // (covered in the next example)
          
          GoRoute(
            path: 'settings',
            name: 'settings',
            pageBuilder: (context, state) {
              return MaterialPage(
                key: state.pageKey,
                child: const SettingsScreen(),
              );
            },
          ),
        ],
      ),
      
      // ============================================
      // ROUTES OUTSIDE THE SHELL
      // ============================================
      // These routes are not wrapped by the shell
      // They cover the entire screen (no bottom nav bar)
      // Use this for full-screen pages like login, onboarding, etc.
      
      GoRoute(
        path: '/login',
        name: 'login',
        pageBuilder: (context, state) {
          return MaterialPage(
            key: state.pageKey,
            child: const LoginScreen(),
          );
        },
      ),
      
      GoRoute(
        path: '/onboarding',
        name: 'onboarding',
        pageBuilder: (context, state) {
          return MaterialPage(
            key: state.pageKey,
            child: const OnboardingScreen(),
          );
        },
      ),
    ],
  );

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'ShellRoute Demo',
      routerConfig: _router,
    );
  }
}

// ============================================
// MAIN SCAFFOLD WITH BOTTOM NAVIGATION
// ============================================
// This is the shell that wraps all the child routes
// It displays the bottom navigation bar and the current child

class MainScaffold extends StatelessWidget {
  final Widget child;
  
  const MainScaffold({super.key, required this.child});

  @override
  Widget build(BuildContext context) {
    // Get the current location to determine the selected tab
    final location = GoRouterState.of(context).uri.path;
    
    // Determine the selected index based on the current location
    int selectedIndex = 0;
    if (location.startsWith('/home')) {
      selectedIndex = 0;
    } else if (location.startsWith('/search')) {
      selectedIndex = 1;
    } else if (location.startsWith('/profile')) {
      selectedIndex = 2;
    }
    
    return Scaffold(
      // The body displays the current child route
      // This changes as the user navigates within the shell
      body: child,
      
      // Bottom navigation bar
      // This remains visible as the user navigates between tabs
      bottomNavigationBar: NavigationBar(
        selectedIndex: selectedIndex,
        onDestinationSelected: (index) {
          // Navigate to the selected tab
          // We use context.go() to replace the current route
          // This maintains the single-route-per-tab pattern
          
          switch (index) {
            case 0:
              context.go('/home');
              break;
            case 1:
              context.go('/search');
              break;
            case 2:
              context.go('/profile');
              break;
          }
        },
        destinations: const [
          NavigationDestination(
            icon: Icon(Icons.home),
            label: 'Home',
          ),
          NavigationDestination(
            icon: Icon(Icons.search),
            label: 'Search',
          ),
          NavigationDestination(
            icon: Icon(Icons.person),
            label: 'Profile',
          ),
        ],
      ),
    );
  }
}

// ============================================
// SCREEN WIDGETS
// ============================================

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
        actions: [
          IconButton(
            icon: const Icon(Icons.settings),
            // Navigate to settings (full-screen, outside the shell)
            onPressed: () => context.push('/settings'),
          ),
          IconButton(
            icon: const Icon(Icons.logout),
            // Navigate to login (full-screen, outside the shell)
            onPressed: () => context.push('/login'),
          ),
        ],
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Product 1'),
            onTap: () {
              // Navigate to product details (within the shell)
              // The bottom navigation bar remains visible
              context.push('/product/1');
            },
          ),
          ListTile(
            title: const Text('Product 2'),
            onTap: () {
              context.push('/product/2');
            },
          ),
          ListTile(
            title: const Text('Product 3'),
            onTap: () {
              context.push('/product/3');
            },
          ),
        ],
      ),
    );
  }
}

class SearchScreen extends StatelessWidget {
  const SearchScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Search'),
        actions: [
          IconButton(
            icon: const Icon(Icons.settings),
            onPressed: () => context.push('/settings'),
          ),
        ],
      ),
      body: const Center(
        child: Text('Search Screen'),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
        actions: [
          IconButton(
            icon: const Icon(Icons.settings),
            onPressed: () => context.push('/settings'),
          ),
        ],
      ),
      body: const Center(
        child: Text('Profile Screen'),
      ),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String productId;
  
  const ProductScreen({super.key, required this.productId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Product $productId'),
        // Add a back button (the bottom nav bar remains visible)
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Product ID: $productId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.pop(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class SettingsScreen extends StatelessWidget {
  const SettingsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Settings'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.pop(),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}

class LoginScreen extends StatelessWidget {
  const LoginScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Login'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('Login Screen'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/home'),
              child: const Text('Login (Go to Home)'),
            ),
          ],
        ),
      ),
    );
  }
}

class OnboardingScreen extends StatelessWidget {
  const OnboardingScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Onboarding'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.go('/home'),
          child: const Text('Get Started'),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **ShellRoute concept**: A `ShellRoute` wraps child routes with a common layout that remains visible while the inner routes change. This is perfect for:
  - Bottom navigation bars
  - Top tabs
  - Navigation drawers
  - Any persistent UI that should stay visible across multiple screens

- **ShellRoute structure**:
  - **`path`**: The base path for all child routes. Child routes are relative to this path. For example, if the shell path is `/` and a child route path is `home`, the full path is `/home`.
  - **`builder`**: Creates the shell widget (e.g., bottom navigation bar). It receives:
    - `context`: BuildContext for building widgets
    - `state`: GoRouterState with navigation information
    - `child`: The widget for the current inner route (this changes as the user navigates)
  - **`routes`**: List of child routes displayed within the shell. These routes are wrapped by the shell.

- **`MainScaffold`**: The shell widget that displays:
  - **`body`**: The `child` parameter from the builder, which is the current inner route. This changes as the user navigates within the shell.
  - **`bottomNavigationBar`**: A persistent bottom navigation bar that allows switching between tabs. The bar remains visible as the user navigates.

- **Determining the selected tab**: We use `GoRouterState.of(context).uri.path` to get the current location and determine which tab is selected. This keeps the bottom navigation bar in sync with the current route.

- **Tab navigation**: When a tab is selected, we use `context.go('/tabPath')` to navigate. This replaces the current route with the selected tab, maintaining the single-route-per-tab pattern (the stack doesn't grow indefinitely).

- **Nested routes within the shell**: Routes like `/product/:id` are child routes of the shell. They are displayed within the shell (the bottom navigation bar remains visible). This allows for deep navigation within a tab while keeping the tab navigation accessible.

- **Routes outside the shell**: Routes like `/login` and `/onboarding` are defined at the top level (outside the shell). They cover the entire screen and don't show the bottom navigation bar. This is useful for full-screen flows that shouldn't have the navigation UI.

- **`pageBuilder` vs. `builder`**:
  - **`builder`**: Creates the widget for the route. Used for simple routes where you don't need custom page configuration.
  - **`pageBuilder`**: Creates a `Page` widget (e.g., `MaterialPage`, `CupertinoPage`) with custom configuration. This allows you to set page transitions, titles, and other page-level properties. `state.pageKey` is used to uniquely identify the page.

- **Navigation within the shell**: Use `context.push()` for routes within the shell (e.g., `/product/:id`). This adds to the stack within the shell, allowing the user to go back while the shell remains visible.

- **Navigation outside the shell**: Use `context.push()` or `context.go()` for routes outside the shell (e.g., `/settings`, `/login`). These routes cover the entire screen and don't show the shell.

### **Multiple Shells for Complex Layouts**

```dart
import 'package:flutter/material.dart';
import 'package:go_router/go_router.dart';

void main() {
  runApp(const MultipleShellsApp());
}

class MultipleShellsApp extends StatelessWidget {
  final GoRouter _router = GoRouter(
    initialLocation: '/home',
    debugLogDiagnostics: true,
    routes: [
      // ============================================
      // OUTER SHELL: DRAWER
      // ============================================
      // This shell wraps the entire app with a navigation drawer
      // All routes are displayed within this shell
      
      ShellRoute(
        path: '/',
        builder: (context, state, child) {
          return OuterShell(child: child);
        },
        routes: [
          // ============================================
          // INNER SHELL: BOTTOM NAVIGATION
          // ============================================
          // This shell wraps its child routes with a bottom navigation bar
          // It's a child of the outer shell, creating nested shells
          
          ShellRoute(
            path: 'tabs',
            builder: (context, state, child) {
              return InnerShell(child: child);
            },
            routes: [
              // Tab 1: Home
              GoRoute(
                path: 'home',
                name: 'home',
                pageBuilder: (context, state) => MaterialPage(
                  key: state.pageKey,
                  child: const HomeScreen(),
                ),
              ),
              
              // Tab 2: Search
              GoRoute(
                path: 'search',
                name: 'search',
                pageBuilder: (context, state) => MaterialPage(
                  key: state.pageKey,
                  child: const SearchScreen(),
                ),
              ),
              
              // Tab 3: Notifications
              GoRoute(
                path: 'notifications',
                name: 'notifications',
                pageBuilder: (context, state) => MaterialPage(
                  key: state.pageKey,
                  child: const NotificationsScreen(),
                ),
              ),
              
              // Nested route within inner shell: Product
              GoRoute(
                path: 'product/:id',
                name: 'product',
                pageBuilder: (context, state) {
                  final productId = state.pathParameters['id'] ?? 'unknown';
                  return MaterialPage(
                    key: state.pageKey,
                    child: ProductScreen(productId: productId),
                  );
                },
              ),
            ],
          ),
          
          // ============================================
          // ROUTES OUTSIDE THE INNER SHELL
          // ============================================
          // These routes are within the outer shell (drawer visible)
          // but outside the inner shell (no bottom nav bar)
          
          GoRoute(
            path: 'settings',
            name: 'settings',
            pageBuilder: (context, state) => MaterialPage(
              key: state.pageKey,
              child: const SettingsScreen(),
            ),
          ),
          
          GoRoute(
            path: 'profile',
            name: 'profile',
            pageBuilder: (context, state) => MaterialPage(
              key: state.pageKey,
              child: const ProfileScreen(),
            ),
          ),
        ],
      ),
      
      // ============================================
      // ROUTES OUTSIDE ALL SHELLS
      // ============================================
      // These routes cover the entire screen
      // No drawer or bottom navigation bar
      
      GoRoute(
        path: '/login',
        name: 'login',
        pageBuilder: (context, state) => MaterialPage(
          key: state.pageKey,
          child: const LoginScreen(),
        ),
      ),
    ],
  );

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Multiple Shells Demo',
      routerConfig: _router,
    );
  }
}

// ============================================
// OUTER SHELL WITH DRAWER
// ============================================
// This shell wraps the entire app with a navigation drawer
// The drawer is accessible from all routes within this shell

class OuterShell extends StatelessWidget {
  final Widget child;
  
  const OuterShell({super.key, required this.child});

  @override
  Widget build(BuildContext context) {
    final location = GoRouterState.of(context).uri.path;
    
    // Determine the selected drawer item
    int drawerSelectedIndex = 0;
    if (location.startsWith('/tabs/home') || location.startsWith('/tabs/search') || location.startsWith('/tabs/notifications')) {
      drawerSelectedIndex = 0;
    } else if (location.startsWith('/settings')) {
      drawerSelectedIndex = 1;
    } else if (location.startsWith('/profile')) {
      drawerSelectedIndex = 2;
    }
    
    return Scaffold(
      // The body displays either the inner shell or a full-screen route
      body: child,
      
      // App bar with drawer menu
      appBar: AppBar(
        title: const Text('Multiple Shells'),
      ),
      
      // Navigation drawer
      drawer: Drawer(
        child: ListView(
          children: [
            DrawerHeader(
              decoration: BoxDecoration(
                color: Theme.of(context).colorScheme.primary,
              ),
              child: const Column(
                crossAxisAlignment: CrossAxisAlignment.start,
                children: [
                  CircleAvatar(
                    radius: 30,
                    child: Icon(Icons.person),
                  ),
                  SizedBox(height: 16),
                  Text(
                    'John Doe',
                    style: TextStyle(color: Colors.white, fontSize: 18),
                  ),
                  Text(
                    'john@example.com',
                    style: TextStyle(color: Colors.white70, fontSize: 14),
                  ),
                ],
              ),
            ),
            ListTile(
              leading: const Icon(Icons.home),
              title: const Text('Home'),
              selected: drawerSelectedIndex == 0,
              onTap: () {
                Navigator.pop(context); // Close drawer
                context.go('/tabs/home');
              },
            ),
            ListTile(
              leading: const Icon(Icons.settings),
              title: const Text('Settings'),
              selected: drawerSelectedIndex == 1,
              onTap: () {
                Navigator.pop(context); // Close drawer
                context.go('/settings');
              },
            ),
            ListTile(
              leading: const Icon(Icons.person),
              title: const Text('Profile'),
              selected: drawerSelectedIndex == 2,
              onTap: () {
                Navigator.pop(context); // Close drawer
                context.go('/profile');
              },
            ),
            const Divider(),
            ListTile(
              leading: const Icon(Icons.logout),
              title: const Text('Logout'),
              onTap: () {
                Navigator.pop(context); // Close drawer
                context.push('/login');
              },
            ),
          ],
        ),
      ),
    );
  }
}

// ============================================
// INNER SHELL WITH BOTTOM NAVIGATION
// ============================================
// This shell wraps its child routes with a bottom navigation bar
// It's displayed within the outer shell (drawer still accessible)

class InnerShell extends StatelessWidget {
  final Widget child;
  
  const InnerShell({super.key, required this.child});

  @override
  Widget build(BuildContext context) {
    final location = GoRouterState.of(context).uri.path;
    
    // Determine the selected bottom nav item
    int selectedIndex = 0;
    if (location.startsWith('/tabs/home')) {
      selectedIndex = 0;
    } else if (location.startsWith('/tabs/search')) {
      selectedIndex = 1;
    } else if (location.startsWith('/tabs/notifications')) {
      selectedIndex = 2;
    }
    
    return Scaffold(
      body: child,
      bottomNavigationBar: NavigationBar(
        selectedIndex: selectedIndex,
        onDestinationSelected: (index) {
          switch (index) {
            case 0:
              context.go('/tabs/home');
              break;
            case 1:
              context.go('/tabs/search');
              break;
            case 2:
              context.go('/tabs/notifications');
              break;
          }
        },
        destinations: const [
          NavigationDestination(
            icon: Icon(Icons.home),
            label: 'Home',
          ),
          NavigationDestination(
            icon: Icon(Icons.search),
            label: 'Search',
          ),
          NavigationDestination(
            icon: Icon(Icons.notifications),
            label: 'Notifications',
          ),
        ],
      ),
    );
  }
}

// ============================================
// SCREEN WIDGETS
// ============================================

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: ListView(
        children: [
          ListTile(
            title: const Text('Product 1'),
            onTap: () => context.push('/tabs/product/1'),
          ),
          ListTile(
            title: const Text('Product 2'),
            onTap: () => context.push('/tabs/product/2'),
          ),
        ],
      ),
    );
  }
}

class SearchScreen extends StatelessWidget {
  const SearchScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return const Scaffold(
      body: Center(child: Text('Search Screen')),
    );
  }
}

class NotificationsScreen extends StatelessWidget {
  const NotificationsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return const Scaffold(
      body: Center(child: Text('Notifications Screen')),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String productId;
  
  const ProductScreen({super.key, required this.productId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Product $productId')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Product ID: $productId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.pop(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class SettingsScreen extends StatelessWidget {
  const SettingsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.pop(),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.pop(),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}

class LoginScreen extends StatelessWidget {
  const LoginScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.go('/tabs/home'),
          child: const Text('Login'),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Nested shells**: You can nest multiple `ShellRoute` instances to create complex layouts. In this example:
  - **Outer shell**: Wraps the entire app with a navigation drawer. All routes are within this shell (drawer is always accessible).
  - **Inner shell**: Wraps its child routes with a bottom navigation bar. This is a child of the outer shell, creating nested navigation UI.

- **Outer shell structure**:
  - Provides a navigation drawer with menu items
  - Displays either the inner shell (for tab routes) or full-screen routes (like settings, profile)
  - The drawer remains accessible from all routes within this shell

- **Inner shell structure**:
  - Provides a bottom navigation bar for switching between tabs
  - Displays tab screens (home, search, notifications)
  - The bottom navigation bar remains visible while navigating within the inner shell

- **Route hierarchy**:
  - `/login`: Outside all shells (full-screen, no drawer or bottom nav)
  - `/tabs/*`: Within both shells (drawer and bottom nav visible)
  - `/settings` or `/profile`: Within outer shell only (drawer visible, no bottom nav)

- **Path structure**:
  - Outer shell path: `/`
  - Inner shell path: `tabs` (relative to outer shell)
  - Tab routes: `home`, `search`, `notifications` (relative to inner shell)
  - Full route paths: `/tabs/home`, `/tabs/search`, `/tabs/notifications`

- **Navigation between shells**:
  - Navigate to `/tabs/home` → Shows both drawer and bottom nav
  - Navigate to `/settings` → Shows drawer only (no bottom nav)
  - Navigate to `/login` → Shows nothing (full-screen)

- **Use cases for multiple shells**:
  - Apps with both a drawer and bottom navigation (e.g., email apps, social media apps)
  - Apps with different navigation patterns for different sections (e.g., e-commerce apps with a drawer for account settings and bottom nav for browsing)
  - Complex apps with nested navigation requirements

---

## **18.4 Redirects and Guards**

GoRouter provides powerful redirect and guard functionality for controlling navigation based on conditions like authentication state.

### **Authentication Redirect**

```dart
import 'package:flutter/material.dart';
import 'package:go_router/go_router.dart';

void main() {
  runApp(const AuthGuardApp());
}

// ============================================
// AUTHENTICATION SERVICE (SIMULATED)
// ============================================
// In a real app, this would communicate with your backend
// and manage authentication state (tokens, sessions, etc.)

class AuthService {
  // Simulated authentication state
  bool _isAuthenticated = false;
  
  // Stream of authentication state changes
  // This allows the router to react to authentication changes
  Stream<bool> get authStateStream => Stream.value(_isAuthenticated);
  
  // Check if the user is authenticated
  bool get isAuthenticated => _isAuthenticated;
  
  // Login method (simulated)
  Future<void> login() async {
    await Future.delayed(const Duration(seconds: 1));
    _isAuthenticated = true;
    // In a real app, you would emit the new state to the stream
    // Or use a state management solution like Provider, Riverpod, Bloc, etc.
  }
  
  // Logout method (simulated)
  Future<void> logout() async {
    await Future.delayed(const Duration(seconds: 1));
    _isAuthenticated = false;
    // In a real app, you would emit the new state to the stream
  }
  
  // Get the current user (simulated)
  String? get currentUser {
    if (_isAuthenticated) {
      return 'John Doe';
    }
    return null;
  }
}

// Global instance of the auth service
// In a real app, you would use dependency injection (e.g., Provider, GetIt)
final authService = AuthService();

class AuthGuardApp extends StatelessWidget {
  // ============================================
  // GOROUTER WITH AUTHENTICATION REDIRECTS
  // ============================================
  
  final GoRouter _router = GoRouter(
    initialLocation: '/',
    debugLogDiagnostics: true,
    
    // Redirect function: called for every navigation attempt
    // This is where you implement guards and redirects
    redirect: (context, state) {
      // Get the current authentication state
      final isAuthenticated = authService.isAuthenticated;
      
      // Get the current location (path)
      final location = state.uri.path;
      
      print('Redirect check: location=$location, authenticated=$isAuthenticated');
      
      // ============================================
      // GUARD: PROTECTED ROUTES
      // ============================================
      // Redirect to login if trying to access protected routes while not authenticated
      
      if (!isAuthenticated && _isProtectedRoute(location)) {
        // Store the intended destination for redirect after login
        // This is passed as a query parameter to the login route
        final redirectTo = state.uri.toString();
        
        print('Redirecting to login: intended destination=$redirectTo');
        
        // Return the new location (redirect target)
        // This causes the router to navigate to the login page instead
        return Uri(path: '/login', queryParameters: {'redirect': redirectTo}).toString();
      }
      
      // ============================================
      // GUARD: LOGIN REDIRECT WHEN AUTHENTICATED
      // ============================================
      // Redirect to home if trying to access login while already authenticated
      
      if (isAuthenticated && location == '/login') {
        print('Redirecting to home: already authenticated');
        
        return '/';
      }
      
      // ============================================
      // GUARD: ONBOARDING REDIRECT
      // ============================================
      // Redirect to onboarding if the user hasn't completed it
      // This is simulated with a flag in the auth service
      
      // In a real app, you might check:
      // - Whether the user has completed onboarding
      // - Whether the user has accepted terms and conditions
      // - Whether the user has set up a profile
      
      // final hasCompletedOnboarding = authService.hasCompletedOnboarding;
      // if (!hasCompletedOnboarding && location != '/onboarding') {
      //   return '/onboarding';
      // }
      
      // No redirect needed
      return null;
    },
    
    // Refresh stream: listen to changes and refresh the router
    // This allows the router to react to authentication state changes
    refreshListenable: authService.authStateStream,
    
    routes: [
      // ============================================
      // PUBLIC ROUTES
      // ============================================
      // These routes are accessible to all users (authenticated or not)
      
      GoRoute(
        path: '/',
        name: 'home',
        builder: (context, state) => const HomeScreen(),
      ),
      
      GoRoute(
        path: '/login',
        name: 'login',
        builder: (context, state) {
          // Get the redirect parameter from the URL
          // This is where the user was trying to go before being redirected to login
          final redirect = state.uri.queryParameters['redirect'];
          
          return LoginScreen(redirectTo: redirect);
        },
      ),
      
      GoRoute(
        path: '/register',
        name: 'register',
        builder: (context, state) => const RegisterScreen(),
      ),
      
      // ============================================
      // PROTECTED ROUTES
      // ============================================
      // These routes require authentication
      // If an unauthenticated user tries to access them, they'll be redirected to login
      
      GoRoute(
        path: '/profile',
        name: 'profile',
        builder: (context, state) => const ProfileScreen(),
      ),
      
      GoRoute(
        path: '/settings',
        name: 'settings',
        builder: (context, state) => const SettingsScreen(),
      ),
      
      GoRoute(
        path: '/orders',
        name: 'orders',
        builder: (context, state) => const OrdersScreen(),
      ),
      
      GoRoute(
        path: '/order/:id',
        name: 'order',
        builder: (context, state) {
          final orderId = state.pathParameters['id'] ?? 'unknown';
          return OrderDetailScreen(orderId: orderId);
        },
      ),
    ],
  );

  // Helper function to check if a route is protected
  bool _isProtectedRoute(String path) {
    // List of protected routes
    const protectedRoutes = {
      '/profile',
      '/settings',
      '/orders',
    };
    
    // Check if the path starts with any protected route
    // This handles both exact matches and nested routes (e.g., /orders, /order/123)
    for (final route in protectedRoutes) {
      if (path == route || path.startsWith('$route/')) {
        return true;
      }
    }
    
    return false;
  }

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Auth Guard Demo',
      routerConfig: _router,
    );
  }
}

// ============================================
// SCREEN WIDGETS
// ============================================

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('Home Screen (Public)'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.push('/profile'),
              child: const Text('Go to Profile (Protected)'),
            ),
            const SizedBox(height: 10),
            ElevatedButton(
              onPressed: () => context.push('/settings'),
              child: const Text('Go to Settings (Protected)'),
            ),
          ],
        ),
      ),
    );
  }
}

class LoginScreen extends StatefulWidget {
  final String? redirectTo;
  
  const LoginScreen({super.key, this.redirectTo});

  @override
  State<LoginScreen> createState() => _LoginScreenState();
}

class _LoginScreenState extends State<LoginScreen> {
  bool _isLoading = false;

  Future<void> _handleLogin() async {
    setState(() {
      _isLoading = true;
    });
    
    try {
      // Call the login method on the auth service
      await authService.login();
      
      // Navigate to the redirect destination or home
      if (mounted) {
        final destination = widget.redirectTo ?? '/';
        context.go(destination);
      }
    } catch (e) {
      // Handle login error
      if (mounted) {
        ScaffoldMessenger.of(context).showSnackBar(
          SnackBar(content: Text('Login failed: $e')),
        );
      }
    } finally {
      if (mounted) {
        setState(() {
          _isLoading = false;
        });
      }
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Login'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            if (widget.redirectTo != null)
              Text('Redirecting to: ${widget.redirectTo}'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: _isLoading ? null : _handleLogin,
              child: _isLoading
                  ? const CircularProgressIndicator()
                  : const Text('Login'),
            ),
          ],
        ),
      ),
    );
  }
}

class RegisterScreen extends StatelessWidget {
  const RegisterScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Register'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.go('/'),
          child: const Text('Go to Home'),
        ),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Profile Screen (Protected)'),
            const SizedBox(height: 10),
            Text('User: ${authService.currentUser}'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () async {
                await authService.logout();
                if (context.mounted) {
                  context.go('/login');
                }
              },
              child: const Text('Logout'),
            ),
          ],
        ),
      ),
    );
  }
}

class SettingsScreen extends StatelessWidget {
  const SettingsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Settings'),
      ),
      body: const Center(
        child: Text('Settings Screen (Protected)'),
      ),
    );
  }
}

class OrdersScreen extends StatelessWidget {
  const OrdersScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Orders'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Order 1'),
            onTap: () => context.push('/order/1'),
          ),
          ListTile(
            title: const Text('Order 2'),
            onTap: () => context.push('/order/2'),
          ),
          ListTile(
            title: const Text('Order 3'),
            onTap: () => context.push('/order/3'),
          ),
        ],
      ),
    );
  }
}

class OrderDetailScreen extends StatelessWidget {
  final String orderId;
  
  const OrderDetailScreen({super.key, required this.orderId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Order $orderId'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Order Detail Screen (Protected)'),
            const SizedBox(height: 10),
            Text('Order ID: $orderId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.pop(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Redirect function**: The `redirect` parameter in `GoRouter` is called for every navigation attempt. It receives the `BuildContext` and `GoRouterState` and returns either `null` (no redirect) or a string (the new location to navigate to). This is where you implement guards and redirects.

- **Authentication guard**: The redirect function checks if the user is authenticated and if they're trying to access a protected route. If not authenticated and accessing a protected route, it redirects to login with the intended destination as a query parameter.

- **`_isProtectedRoute()` helper**: A function that checks if a given path is a protected route. It maintains a list of protected routes and checks both exact matches and nested routes (e.g., `/orders` and `/order/123`).

- **Redirect destination**: When redirecting to login, we pass the intended destination as a query parameter (`redirect`). This allows us to redirect the user back to where they were trying to go after a successful login.

- **`refreshListenable`**: A stream (or `ChangeNotifier`) that the router listens to for changes. When the authentication state changes (e.g., user logs in or out), the router refreshes and re-evaluates the redirect function, potentially causing a redirect.

- **Login flow**:
  1. User tries to access a protected route (e.g., `/profile`)
  2. Redirect function detects that the user is not authenticated
  3. User is redirected to `/login?redirect=/profile`
  4. User logs in successfully
  5. User is redirected to `/profile` (the intended destination)

- **Login screen**: The `LoginScreen` receives the `redirectTo` parameter from the URL. After a successful login, it navigates to the redirect destination or home if none is specified.

- **Logout flow**:
  1. User is authenticated and on a protected route
  2. User clicks logout
  3. Authentication state is updated (via `authService.logout()`)
  4. Router refreshes (via `refreshListenable`)
  5. Redirect function detects that the user is no longer authenticated
  6. User is redirected to `/login`

- **Guard for authenticated users**: The redirect function also checks if an authenticated user is trying to access the login page. If so, it redirects them to home. This prevents authenticated users from seeing the login screen.

- **Additional guards**: You can implement additional guards for:
  - Onboarding completion
  - Terms and conditions acceptance
  - Profile setup
  - Email verification
  - Subscription status
  - Role-based access control (admin-only routes, etc.)

---

## **18.5 Error Handling and 404 Pages**

GoRouter provides built-in error handling and allows you to define custom error pages for different error scenarios.

### **Custom Error Pages**

```dart
import 'package:flutter/material.dart';
import 'package:go_router/go_router.dart';

void main() {
  runApp(const ErrorHandlingApp());
}

class ErrorHandlingApp extends StatelessWidget {
  final GoRouter _router = GoRouter(
    initialLocation: '/',
    debugLogDiagnostics: true,
    
    // ============================================
    // ERROR BUILDER: CUSTOM ERROR PAGE
    // ============================================
    // This is called when navigation fails (e.g., 404, 500, etc.)
    // The error parameter contains details about what went wrong
    
    errorBuilder: (context, state) {
      // state.error contains the error that caused the navigation failure
      // It can be an Exception, a custom error type, or null
      
      final error = state.error;
      
      print('Navigation error: $error');
      
      // You can return different error pages based on the error type
      // For example, 404 vs. 500 vs. network errors
      
      if (error is NotFoundException) {
        return NotFoundScreen(path: state.uri.path);
      } else if (error is UnauthorizedException) {
        return UnauthorizedScreen(path: state.uri.path);
      } else if (error is ServerErrorException) {
        return ServerErrorScreen(error: error);
      } else {
        // Default error page
        return DefaultErrorScreen(error: error);
      }
    },
    
    routes: [
      GoRoute(
        path: '/',
        name: 'home',
        builder: (context, state) => const HomeScreen(),
      ),
      GoRoute(
        path: '/about',
        name: 'about',
        builder: (context, state) => const AboutScreen(),
      ),
      GoRoute(
        path: '/error',
        name: 'error',
        builder: (context, state) {
          // This route intentionally throws an error
          // to demonstrate error handling
          
          final errorType = state.uri.queryParameters['type'] ?? 'default';
          
          switch (errorType) {
            case '404':
              throw NotFoundException('Resource not found');
            case '401':
              throw UnauthorizedException('Unauthorized access');
            case '500':
              throw ServerErrorException('Internal server error');
            default:
              throw Exception('Generic error');
          }
        },
      ),
      GoRoute(
        path: '/product/:id',
        name: 'product',
        builder: (context, state) {
          final productId = state.pathParameters['id'] ?? 'unknown';
          
          // Simulate a product not found error
          if (productId == 'invalid') {
            throw NotFoundException('Product not found: $productId');
          }
          
          return ProductScreen(productId: productId);
        },
      ),
    ],
  );

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Error Handling Demo',
      routerConfig: _router,
    );
  }
}

// ============================================
// CUSTOM EXCEPTION TYPES
// ============================================
// Define custom exception types for different error scenarios
// This allows you to handle them differently in the error builder

class NotFoundException implements Exception {
  final String message;
  
  NotFoundException(this.message);
  
  @override
  String toString() => 'NotFoundException: $message';
}

class UnauthorizedException implements Exception {
  final String message;
  
  UnauthorizedException(this.message);
  
  @override
  String toString() => 'UnauthorizedException: $message';
}

class ServerErrorException implements Exception {
  final String message;
  
  ServerErrorException(this.message);
  
  @override
  String toString() => 'ServerErrorException: $message';
}

// ============================================
// ERROR SCREENS
// ============================================

class NotFoundScreen extends StatelessWidget {
  final String path;
  
  const NotFoundScreen({super.key, required this.path});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Not Found'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(
              Icons.error_outline,
              size: 64,
              color: Colors.red,
            ),
            const SizedBox(height: 16),
            const Text(
              '404',
              style: TextStyle(fontSize: 48, fontWeight: FontWeight.bold),
            ),
            const SizedBox(height: 16),
            Text('Page not found: $path'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}

class UnauthorizedScreen extends StatelessWidget {
  final String path;
  
  const UnauthorizedScreen({super.key, required this.path});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Unauthorized'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(
              Icons.lock,
              size: 64,
              color: Colors.orange,
            ),
            const SizedBox(height: 16),
            const Text(
              '401',
              style: TextStyle(fontSize: 48, fontWeight: FontWeight.bold),
            ),
            const SizedBox(height: 16),
            Text('Unauthorized access: $path'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.push('/login'),
              child: const Text('Login'),
            ),
            const SizedBox(height: 10),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}

class ServerErrorScreen extends StatelessWidget {
  final ServerErrorException error;
  
  const ServerErrorScreen({super.key, required this.error});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Server Error'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(
              Icons.warning,
              size: 64,
              color: Colors.red,
            ),
            const SizedBox(height: 16),
            const Text(
              '500',
              style: TextStyle(fontSize: 48, fontWeight: FontWeight.bold),
            ),
            const SizedBox(height: 16),
            Text(error.message),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}

class DefaultErrorScreen extends StatelessWidget {
  final Object? error;
  
  const DefaultErrorScreen({super.key, this.error});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Error'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(
              Icons.error,
              size: 64,
              color: Colors.red,
            ),
            const SizedBox(height: 16),
            const Text(
              'An Error Occurred',
              style: TextStyle(fontSize: 24, fontWeight: FontWeight.bold),
            ),
            const SizedBox(height: 16),
            Text(error?.toString() ?? 'Unknown error'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.go('/'),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}

// ============================================
// SCREEN WIDGETS
// ============================================

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('About'),
            onTap: () => context.push('/about'),
          ),
          const Divider(),
          ListTile(
            title: const Text('Trigger 404 Error'),
            subtitle: const Text('NotFoundException'),
            onTap: () => context.push('/error?type=404'),
          ),
          ListTile(
            title: const Text('Trigger 401 Error'),
            subtitle: const Text('UnauthorizedException'),
            onTap: () => context.push('/error?type=401'),
          ),
          ListTile(
            title: const Text('Trigger 500 Error'),
            subtitle: const Text('ServerErrorException'),
            onTap: () => context.push('/error?type=500'),
          ),
          ListTile(
            title: const Text('Trigger Generic Error'),
            subtitle: const Text('Exception'),
            onTap: () => context.push('/error'),
          ),
          const Divider(),
          ListTile(
            title: const Text('Product 1 (Valid)'),
            onTap: () => context.push('/product/1'),
          ),
          ListTile(
            title: const Text('Product Invalid (404)'),
            subtitle: const Text('Will throw NotFoundException'),
            onTap: () => context.push('/product/invalid'),
          ),
        ],
      ),
    );
  }
}

class AboutScreen extends StatelessWidget {
  const AboutScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('About'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () => context.go('/'),
          child: const Text('Go Home'),
        ),
      ),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String productId;
  
  const ProductScreen({super.key, required this.productId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Product $productId'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Product ID: $productId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => context.pop(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`errorBuilder`**: The `errorBuilder` parameter in `GoRouter` is called when navigation fails. It receives the `BuildContext` and `GoRouterState`, which contains the `error` property. The error can be an `Exception`, a custom error type, or `null`. You return a custom error screen widget based on the error type.

- **Custom exception types**: By defining custom exception types (`NotFoundException`, `UnauthorizedException`, `ServerErrorException`), you can handle different error scenarios differently. This allows you to show appropriate error pages and provide relevant actions to the user.

- **Error page variations**:
  - **`NotFoundScreen`**: Shown for 404 errors (page not found). Displays the path that was not found and provides a button to go home.
  - **`UnauthorizedScreen`**: Shown for 401 errors (unauthorized access). Displays the path and provides buttons to login or go home.
  - **`ServerErrorScreen`**: Shown for 500 errors (server errors). Displays the error message and provides a button to go home.
  - **`DefaultErrorScreen`**: Shown for all other errors. Displays the error details and provides a button to go home.

- **Triggering errors intentionally**: The `/error` route demonstrates error handling by throwing different types of exceptions based on a query parameter. This allows you to test different error scenarios.

- **Error handling in route builders**: You can also throw errors from route builders. For example, the `/product/:id` route throws a `NotFoundException` if the product ID is invalid. This causes the `errorBuilder` to be called with the error.

- **Error page navigation**: Error pages typically provide navigation options to help the user recover from the error. Common options include:
  - Go back to the previous screen
  - Go home
  - Login (for 401 errors)
  - Retry the action

- **Logging errors**: The `errorBuilder` is a good place to log errors to your analytics service (e.g., Firebase Crashlytics, Sentry). This helps you track and diagnose issues in production.

- **User-friendly error messages**: When displaying errors to users, avoid showing technical error messages. Instead, show user-friendly messages that explain what went wrong and what the user can do about it.

---

## **Chapter Summary**

In this chapter, we covered GoRouter and advanced navigation patterns in Flutter:

### **Key Takeaways:**

1. **GoRouter Overview**:
   - Declarative routing library built on Navigator 2.0
   - Simplifies Navigator 2.0 with a cleaner API
   - Automatic URL parsing and updating
   - Built-in deep linking support
   - Less boilerplate code than raw Navigator 2.0

2. **GoRouter Configuration**:
   - `initialLocation`: The route the app starts on
   - `debugLogDiagnostics`: Enable logging for debugging
   - `errorBuilder`: Custom error page for navigation failures
   - `routes`: List of `GoRoute` objects defining all routes

3. **GoRoute Properties**:
   - `path`: URL path that maps to this route
   - `name`: Optional name for programmatic navigation
   - `builder` or `pageBuilder`: Function that builds the widget/page
   - `parentBuilder`: Optional parent page builder for nested routes

4. **Navigation Methods**:
   - `context.go()`: Replace current route in the stack
   - `context.goNamed()`: Replace using route name
   - `context.push()`: Push new route onto the stack
   - `context.pushNamed()`: Push using route name
   - `context.pop()`: Pop current route (optionally with result)

5. **Path and Query Parameters**:
   - Path parameters: Defined with `:` in the route path (e.g., `:id`)
   - Query parameters: Defined in the URL after `?`
   - Access via `state.pathParameters` and `state.uri.queryParameters`

6. **ShellRoute**:
   - Wraps child routes with a common layout
   - Perfect for bottom navigation, tabs, drawers
   - Shell remains visible while inner routes change
   - Can nest multiple shells for complex layouts

7. **Redirects and Guards**:
   - `redirect` function: Called for every navigation attempt
   - Implement authentication guards, role-based access control
   - Redirect to login for protected routes
   - Use `refreshListenable` to react to state changes

8. **Error Handling**:
   - `errorBuilder`: Custom error page for navigation failures
   - Define custom exception types for different error scenarios
   - Show appropriate error pages and recovery actions
   - Log errors to analytics services

### **Next Steps:**

Now that you understand GoRouter and advanced navigation, the next chapters will cover:

- **Chapter 19: Networking & Data**: HTTP requests, REST APIs, JSON serialization
- **Chapter 20: Authentication & Security**: OAuth 2.0, JWT tokens, secure storage
- **Chapter 21: GraphQL & WebSockets**: GraphQL client setup, real-time data with WebSockets

---

**End of Chapter 18**